## Sequence to sequence language translation using Gemma  
In here we have used the following to train Gemma to understand the south Indian language called "Malayalam."  

Tools and libraries used: **Pytorch, bitsandbytes, Huggingface, Lora, and Pfet.**  

One of the unique properties of this training is the dataset.  
This is an English-Malayalam parallel corpus which contains around 4 lakh parallel corpora. 
English sentences are from the COCO dataset and are translated using the Google API. You can find the trainable text files at: 

[English-Malayalam Translation Corpus](https://github.com/eksubin/English-Malayalam-Translation-Corpus?tab=readme-ov-file).  

If you are doubtful about training Gemma using Pytorch, please refer to my previous notebooks on inferencing and fine-tuning Gemma using Pytorch:  
- [Understanding Gemma Fine-tuning Using Pytorch](https://www.kaggle.com/code/subinek/understanding-gemma-finetuning-using-pytorch)  
- [Gemma Inferencing with Pytorch](https://www.kaggle.com/code/subinek/gemma-inferencing-with-pytorch)

Please upvote this notebook if you find it interesting

### Install all the required libraries
- HuggingFace : For downloading the Gemma version trainable using pytorch
- bitsandbytes : Loading the configurations for training
- peft : Parameter efficient finetuning

In [1]:
#!pip install -U trl
!pip install git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-4s_zavbk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-4s_zavbk
  Resolved https://github.com/huggingface/trl.git to commit d6a8f2c2f6780affae11e8fbab6101ceeebaf0ca
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 69.0 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.13.0.dev0-py3-none-any.whl size=319904 sha256=ddc3cc2653e61fc91530e5fc8c7d3b6745b1c790ff238e43709cdb71144c19e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-as5era_r/wheels/22/0e/42/319b77b2648bb6140ef2b08b0478ede9ca3cc7879fcd022d36
Successfully built trl
  Attempting uninstall: transformers
    Found existing installation: transform

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.0 MB/s eta 0:00:00


In [3]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.2 MB/s eta 0:00:00


In [4]:
!pip install ipywidgets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 

In [5]:
import os
import wandb
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset

user_secrets = UserSecretsClient()
huggingface_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

login(token=huggingface_token)
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.init(project="gemma2", name="malayalamTranslation")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Currently logged in as: subinek. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241129_205650-uw0jkd0i
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run malayalamTranslation
wandb: ⭐️ View project at https://wandb.ai/subinek/gemma2
wandb: 🚀 View run at https://wandb.ai/subinek/gemma2/runs/uw0jkd0i


### Load the model
* Provided the model ID to download it from hugging face.
* `BitsAndBytesConfig` : Is a library used for efficient model quantization. In here we have quantized the model to 4 bit precision.
* `AutoTokenizer` : Loads the tokenizer specific to the `model_id`
* `AutoModelForCausalLM` : Load the model with the quantization configs

In [6]:
# Load the model and the tokenizers
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Inferencing the Gemma Model
- Conduct a test inferencing using the model.

In [7]:
# Perform inference with the Gemma model
text = "Quote: Imagination is more"  # Input text for the model
device = "cuda:0"  # Specify the device to use (GPU)

# Tokenize the input text and convert it to tensor format, moving it to the specified device
inputs = tokenizer(text, return_tensors="pt").to(device)

# Generate output from the model with a limit on the number of new tokens
outputs = model.generate(**inputs, max_new_tokens=20)

# Decode the generated tokens back to text and print the result, skipping special tokens
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

- Albert Einstein

The


### Training Gemma Using Malayalam
- Download the Malayalam to English translation corpus from the provided GitHub link.
- The dataset contains 400,000 samples of translation from Malayalam to English.
- Created using the Google Translate API.

In [8]:
# Get the english to malayalam corpus
!wget https://raw.githubusercontent.com/eksubin/English-Malayalam-Translation-Corpus/refs/heads/main/english.txt
!wget https://raw.githubusercontent.com/eksubin/English-Malayalam-Translation-Corpus/refs/heads/main/malayalam.txt

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-11-29 20:59:08--  https://raw.githubusercontent.com/eksubin/English-Malayalam-Translation-Corpus/refs/heads/main/english.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19281987 (18M) [text/plain]
Saving to: 'english.txt'

english.txt         100%[===================>]  18.39M  36.5MB/s    in 0.5s    

2024-11-29 20:59:10 (36.5 MB/s) - 'english.txt' saved [19281987/19281987]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-11-29 20:59:12--  https://raw.githubusercontent.com/eksubin/English-Malayalam-Translation-Corpus/refs/heads/main/malayalam.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58141526 (55M) [application/octet-stream]
Saving to: 'malayalam.txt'

malayalam.txt       100%[===================>]  55.45M  61.1MB/s    in 0.9s    

2024-11-29 20:59:17 (61.1 MB/s) - 'malayalam.txt' saved [58141526/58141526]



In [9]:
# Test samples from the dataset
with open('/kaggle/working/english.txt', 'r', encoding='utf-8') as en_file:
    english_lines = en_file.readlines()

with open('/kaggle/working/malayalam.txt', 'r', encoding='utf-8') as ml_file:
    malayalam_lines = ml_file.readlines()

# Example: Print the first 5 lines
print('English:', english_lines[:5])
print('Malayalam:', malayalam_lines[:5])

English: ['A large freight train sits in a train station.\n', 'People boat down a river with flags strung across it.\n', 'A set of railway tracks as seen from a train.\n', 'A canoe with a motor speeding down a river.\n', 'A yellow van with people trying to display something in an empty square.\n']
Malayalam: ['ഒരു വലിയ ചരക്ക് ട്രെയിൻ ഒരു ട്രെയിൻ സ്റ്റേഷനിൽ ഇരിക്കുന്നു.\n', 'ആളുകൾ ഒരു നദിക്കരയിൽ പതാകകൾ പതിച്ചിട്ടുണ്ട്.\n', 'ട്രെയിനിൽ നിന്ന് കാണുന്നതുപോലെ ഒരു കൂട്ടം റെയിൽവേ ട്രാക്കുകൾ.\n', 'ഒരു നദിയിലൂടെ മോട്ടോർ വേഗതയുള്ള ഒരു പീരങ്കി.\n', 'ശൂന്യമായ സ്ക്വയറിൽ എന്തെങ്കിലും പ്രദർശിപ്പിക്കാൻ ശ്രമിക്കുന്ന ആളുകളുള്ള ഒരു മഞ്ഞ വാൻ.\n']


In [10]:
# Load your Malayalam-to-English dataset
def load_data(malayalam_file, english_file):
    with open(malayalam_file, "r", encoding="utf-8") as m_file, open(english_file, "r", encoding="utf-8") as e_file:
        malayalam_lines = m_file.readlines()
        english_lines = e_file.readlines()

    # Ensure both files have the same number of lines
    assert len(malayalam_lines) == len(english_lines), "Mismatch in line counts between source and target files."
    
    # Create a Dataset object
    data = {"source": malayalam_lines, "target": english_lines}
    return Dataset.from_dict(data)

# Paths to your training data
malayalam_file = "malayalam.txt"
english_file = "english.txt"

# Load data
dataset = load_data(malayalam_file, english_file)

# Split into training and validation sets
data = dataset.train_test_split(test_size=0.01)

In [11]:
# Provide the LORA config details
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

# Formatting function for Malayalam-to-English translation
def formatting_func(example):
    # Tokenize the source (Malayalam) and target (English) texts
    inputs = tokenizer(example["source"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(example["target"], max_length=128, truncation=True, padding="max_length")
    
    # Return tokenized inputs and labels
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

# Map the formatting function over the dataset
tokenized_data = data.map(formatting_func, batched=True, remove_columns=["source", "target"])


# Training arguments for fine-tuning
sft_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=10,
    max_steps=100,
    learning_rate=3e-5,
    fp16=True,  # Mixed-precision training for efficiency
    logging_steps=10,
    output_dir="outputs",
    save_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=3,
)


# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=sft_arguments,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

Map:   0%|          | 0/358213 [00:00<?, ? examples/s]

Map:   0%|          | 0/3619 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/2435954604.py:44: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:298: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'r

Step,Training Loss,Validation Loss
50,2.593900,2.569399
100,2.453300,2.410065


TrainOutput(global_step=100, training_loss=2.6951832008361816, metrics={'train_runtime': 1405.8163, 'train_samples_per_second': 1.138, 'train_steps_per_second': 0.071, 'total_flos': 2447388966912000.0, 'train_loss': 2.6951832008361816, 'epoch': 0.004466603761997019})

In [12]:
# Save the model and tokenizer
model.save_pretrained("custom_model.ckpt")
tokenizer.save_pretrained("custom_tokenizer.model")

('custom_tokenizer.model/tokenizer_config.json',
 'custom_tokenizer.model/special_tokens_map.json',
 'custom_tokenizer.model/tokenizer.model',
 'custom_tokenizer.model/added_tokens.json',
 'custom_tokenizer.model/tokenizer.json')

### Inferencing the trained model

In [13]:
# Function to translate Malayalam text to English
def translate_malayalam_to_english(text, tokenizer=tokenizer, model=model, max_length=128):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True)
    
    # Generate translation using the model
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=4,  # Beam search for better translations
        early_stopping=True
    )
    
    # Decode the output and return the English translation
    translation = tokenizer.decode(output[0], skip_special_tokens=True)
    return translation

# Example Malayalam text for testing
malayalam_text = "ഞാൻ ഇന്ന് രാവിലെ ഉണർന്നു. ഒരു ചായ കുടിച്ചു"  # "What is your name?" in Malayalam

# Get the translation
english_translation = translate_malayalam_to_english(malayalam_text, tokenizer, model)

# Print the results
print(f"Malayalam: {malayalam_text}")
print(f"English: {english_translation}")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Malayalam: ഞാൻ ഇന്ന് രാവിലെ ഉണർന്നു. ഒരു ചായ കുടിച്ചു
English: ഞാൻ ഇന്ന് രാവിലെ ഉണർന്നു. ഒരു ചായ കുടിച്ചു.
I woke up this morning. I had a cup of tea.
I woke up this morning. I had a cup of tea.
I woke up this morning. I had a cup of tea.
I woke up this morning. I had a cup of tea.
I woke up this morning. I had a cup of tea.


# Thank You!

Thank you for taking the time to go through my Kaggle notebook! I appreciate your interest and support.

If you found the content helpful or enjoyable, please consider upvoting it. Your positive feedback means a lot to me!

Happy coding!

Best,  
Subin Erattakulangara